In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/mai-ml-decision-trees/train.csv
/kaggle/input/mai-ml-decision-trees/test.csv


In [2]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.metrics as sm
import warnings

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import LabelEncoder

from tqdm import tqdm

warnings.filterwarnings("ignore")

%matplotlib inline

In [3]:
train_df = pd.read_csv("/kaggle/input/mai-ml-decision-trees/train.csv")
test_df = pd.read_csv("/kaggle/input/mai-ml-decision-trees/test.csv")

train_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1
1,1249,3,"Lockyer, Mr. Edward",male,NaN,0,0,1222,7.8792,NaN,S,0
2,1240,2,"Giles, Mr. Ralph",male,24.0,0,0,248726,13.5000,NaN,S,0
3,221,3,"Sunderland, Mr. Victor Francis",male,16.0,0,0,SOTON/OQ 392089,8.0500,NaN,S,1
4,1105,2,"Howard, Mrs. Benjamin (Ellen Truelove Arman)",female,60.0,1,0,24065,26.0000,NaN,S,0


In [4]:
train_df["Title"] = train_df.Name.apply(lambda x: x.split(" ")[1].replace(".", ""))
test_df["Title"] = test_df.Name.apply(lambda x: x.split(" ")[1].replace(".", ""))

title_stat = train_df.groupby("Title").Name.agg(["count"])
title_stat["is_rare"] = title_stat["count"] < 10
titles = title_stat[~title_stat.is_rare].index.values

train_df.Title = train_df.Title.apply(lambda x: x if x in titles else "rare_title")
test_df.Title = test_df.Title.apply(lambda x: x if x in titles else "rare_title")

train_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived,Title
0,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S,1,Miss
1,1249,3,"Lockyer, Mr. Edward",male,NaN,0,0,1222,7.8792,NaN,S,0,Mr
2,1240,2,"Giles, Mr. Ralph",male,24.0,0,0,248726,13.5000,NaN,S,0,Mr
3,221,3,"Sunderland, Mr. Victor Francis",male,16.0,0,0,SOTON/OQ 392089,8.0500,NaN,S,1,Mr
4,1105,2,"Howard, Mrs. Benjamin (Ellen Truelove Arman)",female,60.0,1,0,24065,26.0000,NaN,S,0,Mrs


In [5]:
train_df.groupby("Title").PassengerId.count()

Title
Master         38
Miss          163
Mr            487
Mrs           123
rare_title     39
Name: PassengerId, dtype: int64

In [6]:
train_df["Children"] = train_df.Age.apply(lambda x: 1 if x < 18 else 0)
train_df["Mother"] = train_df.apply(lambda x: 1 if x.Sex == "female" and x.Age > 18 and x.Parch > 0 else 0, axis=1)
train_df["Family_size"] = train_df.SibSp + train_df.Parch + 1

test_df["Children"] = test_df.Age.apply(lambda x: 1 if x < 18 else 0)
test_df["Mother"] = test_df.apply(lambda x: 1 if x.Sex == "female" and x.Age > 18 and x.Parch > 0 else 0, axis=1)
test_df["Family_size"] = test_df.SibSp + test_df.Parch + 1

train_df["Deck"] = train_df.Cabin.astype("str").apply(lambda x: x[0] if x != "nan" else "no_deck")
test_df["Deck"] = test_df.Cabin.astype("str").apply(lambda x: x[0] if x != "nan" else "no_deck")

In [7]:
train_df.Age = train_df.Age.fillna(train_df.Age.dropna().median()).astype("int16")
train_df.Fare = train_df.Fare.fillna(train_df.Fare.dropna().median())
train_df.Embarked = train_df.Embarked.fillna("no_info")

test_df.Age = test_df.Age.fillna(train_df.Age.dropna().median()).astype("int16")
test_df.Fare = test_df.Fare.fillna(train_df.Fare.dropna().median())
test_df.Embarked = test_df.Embarked.fillna("no_info")

In [8]:
train_df.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived,Title,Children,Mother,Family_size,Deck
0,888,1,"Graham, Miss. Margaret Edith",female,19,0,0,112053,30.0000,B42,S,1,Miss,0,0,1,B
1,1249,3,"Lockyer, Mr. Edward",male,28,0,0,1222,7.8792,NaN,S,0,Mr,0,0,1,no_deck
2,1240,2,"Giles, Mr. Ralph",male,24,0,0,248726,13.5000,NaN,S,0,Mr,0,0,1,no_deck
3,221,3,"Sunderland, Mr. Victor Francis",male,16,0,0,SOTON/OQ 392089,8.0500,NaN,S,1,Mr,1,0,1,no_deck
4,1105,2,"Howard, Mrs. Benjamin (Ellen Truelove Arman)",female,60,1,0,24065,26.0000,NaN,S,0,Mrs,0,0,2,no_deck


In [9]:
train_df.Embarked.unique()

array(['S', 'C', 'Q'], dtype=object)

In [10]:
train_v1 = train_df.copy()
test_v1 = test_df.copy()

unnecessary_cols = ["Name", "Ticket", "SibSp", "Parch", "Cabin"]

train_v1 = train_v1.drop(unnecessary_cols, axis=1)
test_v1 = test_v1.drop(unnecessary_cols, axis=1)

encoder = LabelEncoder()
encoding_cols = ["Sex", "Embarked", "Title", "Deck"]

full_df = pd.concat([train_v1, test_v1], axis=0)

for col in encoding_cols:
    full_df[col] = encoder.fit_transform(full_df[col])
    
train_v1 = full_df[full_df.Survived.notnull()]
train_v1.Survived = train_v1.Survived.astype("int8")

test_v1 = full_df[full_df.Survived.isnull()]
test_v1.drop("Survived", axis=1, inplace=True)

feature_cols = [col for col in train_v1.columns if col not in {"PassengerId", "Survived"}]
target_col = "Survived"

X = train_v1[feature_cols].values
y = train_v1[target_col]

In [11]:
train_v1.head()

,Age,Children,Deck,Embarked,Family_size,Fare,Mother,PassengerId,Pclass,Sex,Survived,Title
0,19,0,1,2,1,30.0000,0,888,1,0,1,1
1,28,0,8,2,1,7.8792,0,1249,3,1,0,2
2,24,0,8,2,1,13.5000,0,1240,2,1,0,2
3,16,1,8,2,1,8.0500,0,221,3,1,1,2
4,60,0,8,2,2,26.0000,0,1105,2,0,0,3


In [12]:
from sklearn.tree import DecisionTreeClassifier

RANDOM_STATE = 42

model = DecisionTreeClassifier(max_depth=2, random_state=RANDOM_STATE)
model.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=42, splitter='best')

In [13]:
def make_submission(ids, preds, output_path='submission.csv'):
    subm = pd.DataFrame()
    subm['PassengerId'] = ids
    subm['Survived'] = preds
    subm.to_csv(output_path, index=False)

In [14]:
X_test = test_v1[feature_cols].values
preds = model.predict(X_test)
ids = test_df.PassengerId.values
# make_submission(ids,preds)

In [15]:
from sklearn.model_selection import GridSearchCV

tree_params = {
    'max_depth': list(range(1, 10)),
    'min_samples_leaf': list(range(1, 8))
}
gs = GridSearchCV(
    DecisionTreeClassifier(random_state=RANDOM_STATE),
    param_grid=tree_params,
    scoring='accuracy',
    cv = 5
)
gs.fit(X, y)

print(gs.best_params_)
print(gs.best_score_)

{'max_depth': 3, 'min_samples_leaf': 6}
0.7941176470588235


In [16]:
preds = gs.predict(X_test)
# make_submission(ids,preds)

In [17]:
unnecessary_cols = ["Name", "Ticket", "SibSp", "Parch", "Cabin"]

train_v2 = train_df.drop(unnecessary_cols, axis=1).copy()
test_v2 = test_df.drop(unnecessary_cols, axis=1).copy()

train_v2.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked,Survived,Title,Children,Mother,Family_size,Deck
0,888,1,female,19,30.0000,S,1,Miss,0,0,1,B
1,1249,3,male,28,7.8792,S,0,Mr,0,0,1,no_deck
2,1240,2,male,24,13.5000,S,0,Mr,0,0,1,no_deck
3,221,3,male,16,8.0500,S,1,Mr,1,0,1,no_deck
4,1105,2,female,60,26.0000,S,0,Mrs,0,0,2,no_deck


In [18]:
"""
train и test могут отличаться по значениям категориальных признаков.
Поэтому может получиться так, что train и test будут отличаться по
количеству и названию колонок после преобразования.
Чтобы этого избежать, объединим train и test, выполним OHE, а потом
разъединим обратно.
"""

categorical_cols = ['Pclass', 'Sex', 'Embarked', 'Title', 'Mother', 'Deck']
full_df = pd.concat([train_v2, test_v2], axis=0)
dfs = []

for col in categorical_cols:
    print(f"Encoding {col}")
    df = pd.get_dummies(full_df[col], prefix=col)
    dfs.append(df)
    
full_df = pd.concat([full_df] + dfs, axis=1)
full_df = full_df.drop(categorical_cols, axis=1)

train_v2 = full_df[full_df.Survived.notnull()]
train_v2.Survived = train_v1.Survived.astype("int8")

test_v2 = full_df[full_df.Survived.isnull()]
test_v2 = test_v2.drop("Survived", axis=1)

Encoding Pclass
Encoding Sex
Encoding Embarked
Encoding Title
Encoding Mother
Encoding Deck


In [19]:
set(train_v2.columns) - set(test_v2.columns)

{'Survived'}

In [20]:
feature_cols = [col for col in train_v2.columns if col not in {"PassengerId", "Survived"}]
target_col = "Survived"

X = train_v2[feature_cols].values
y = train_v2[target_col]

In [21]:
tree_params = {
    'max_depth': list(range(1, 10)),
    'min_samples_leaf': list(range(1, 10))
}
gs = GridSearchCV(
    DecisionTreeClassifier(random_state=RANDOM_STATE),
    param_grid=tree_params,
    scoring='accuracy',
    cv = 5
)
gs.fit(X, y)

print(gs.best_params_)
print(gs.best_score_)
# Your code here

{'max_depth': 3, 'min_samples_leaf': 9}
0.8011764705882353


In [22]:
X_test = test_v2[feature_cols]
preds  = gs.predict(X_test)
ids = test_df.PassengerId.values
make_submission(ids, preds)